In [1]:
import os
import time
import torchvision
import torch
import matplotlib.pyplot as plt
import numpy as np
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader

data_dir = 'D:\Code\python\dog breed'

In [ ]:
def read_csv_labels(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()[1:]
    content = [l.rstrip().split(',') for l in lines]
    return dict(((img, label) for img, label in content))

labels = read_csv_labels(os.path.join(data_dir, 'labels.csv'))
print(len(labels))                  # 样本个数
print(len(set(labels.values())))    # 类别

根据`valid_ratio`比例从原有训练集中抽取样本作为验证集，验证集中，同类样本会被放在目录文件夹下

In [3]:
# def copyfile(filename, target_dir):
#     os.makedirs(target_dir, exist_ok=True)
#     shutil.copy(filename, target_dir)

# def reorg_train_valid(data_dir, labels, valid_ratio):
#     """将验证集从原始的训练集中拆分出来"""
#     # n = 训练数据集中样本最少的类别中的样本数 ?
#     n = collections.Counter(labels.values()).most_common()[-1][1]
#     # 验证集中每个类别的样本数
#     n_valid_per_label = max(1, int(n * valid_ratio))
#     label_count = {}
#     for train_file in os.listdir(os.path.join(data_dir, 'train')):
#         # train_file is "xxx.jpg"
#         # train_file.split('.')[0] is "xxx"
#         label = labels[train_file.split('.')[0]]
#         # label 是通过"xxx"找到的字典的value
#         fname = os.path.join(data_dir, 'train', train_file)

#         # 所有图像都放进train_valid
#         copyfile(fname, os.path.join(data_dir, 'train_valid_test',
#                                      'train_valid', label))
        
#         # 如果当前标签没有文件夹或个数不够n，放进valid
#         if label not in label_count or label_count[label] < n_valid_per_label:
#             copyfile(fname, os.path.join(data_dir, 'train_valid_test',
#                                          'valid', label))
#             label_count[label] = label_count.get(label, 0) + 1

#         # 否则放进train
#         else:
#             copyfile(fname, os.path.join(data_dir, 'train_valid_test',
#                                          'train', label))
#     return n_valid_per_label

# #@save
# # test文件夹中的图像统一放入test\unknown
# def reorg_test(data_dir):
#     """在预测期间整理测试集，以方便读取"""
#     for test_file in os.listdir(os.path.join(data_dir, 'test')):
#         copyfile(os.path.join(data_dir, 'test', test_file),
#                  os.path.join(data_dir, 'train_valid_test', 'test',
#                               'unknown'))
        


In [4]:
# def reorg_dog_data(data_dir, valid_ratio):
#     labels = read_csv_labels(os.path.join(data_dir, 'labels.csv'))
#     reorg_train_valid(data_dir, labels, valid_ratio)
#     reorg_test(data_dir)

# valid_ratio = 0.1
# reorg_dog_data(data_dir, valid_ratio)

进行图像增广

In [5]:
transform_train = torchvision.transforms.Compose([
    # 随机裁切
    torchvision.transforms.RandomResizedCrop(224),
    # 翻转
    torchvision.transforms.RandomHorizontalFlip(),
    # 改变亮度、对比度、饱和度
    # 1-brightness 到 1+brightness，其余同理
    torchvision.transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])
])

# 测试集仅进行裁切
transform_test = torchvision.transforms.Compose([
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])
])

读取数据集

In [6]:
# data_dir = 'D:\Code\python\dog breed'
# batch_size = 128

# train_ds, train_valid_ds = [torchvision.datasets.ImageFolder(
#     os.path.join(data_dir, 'train_valid_test', folder),
#     transform=transform_train) for folder in ['train', 'train_valid']]

# valid_ds, test_ds = [torchvision.datasets.ImageFolder(
#     os.path.join(data_dir, 'train_valid_test', folder),
#     transform=transform_test) for folder in ['valid', 'test']]

# train_iter, train_valid_iter = [torch.utils.data.DataLoader(
#     dataset, batch_size, shuffle=True, drop_last=True)
#     for dataset in (train_ds, train_valid_ds)]

# # for batch_idx, (inputs, labels) in enumerate(train_iter):
# #     print(f"Batch {batch_idx+1}:")
# #     print(f"Inputs: {inputs.shape}")
# #     print(f"Labels: {labels.shape}")

# #     break

# valid_iter = torch.utils.data.DataLoader(valid_ds, batch_size, shuffle=False,
#                                          drop_last=True)

# test_iter = torch.utils.data.DataLoader(test_ds, batch_size, shuffle=False,
#                                         drop_last=False)

In [7]:
train_set = datasets.ImageFolder(
    os.path.join(data_dir, 'train_valid_test', 'train'),
    transform=transform_train
)

train_valid_set = datasets.ImageFolder(
    os.path.join(data_dir, 'train_valid_test', 'train_valid'),
    transform=transform_train
)

valid_set = datasets.ImageFolder(
    os.path.join(data_dir, 'train_valid_test', 'valid'),
    transform=transform_test
)

test_set = datasets.ImageFolder(
    os.path.join(data_dir, 'train_valid_test', 'test'),
    transform=transform_test
)

In [8]:
batch_size = 128

train_loader = DataLoader(train_set, batch_size, shuffle=True)
train_valid_loader = DataLoader(train_valid_set, batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size, shuffle=False)

# for batch_idx, (inputs, labels) in enumerate(train_loader):
#     print(f"Batch {batch_idx+1}:")
#     print(f"Inputs: {inputs}")
#     print(f"Labels: {labels}")

#     break

构建训练模型，采用resnet50

In [9]:
los = []        #画图用
v_los = []
timecost = []



In [10]:
class ResNet34(nn.Module):
    def __init__(self):
        super(ResNet34, self).__init__()
        # self.features = torchvision.models.resnet34(pretrained=True)
        self.features = torchvision.models.resnet34()
        self.output_new = nn.Sequential(nn.Linear(1000, 256),
                                       nn.ReLU(),
                                       nn.Linear(256,120)
                                    ) # 1000 512 relu 120
        # for param in self.features.parameters():
        #     param.requires_grad = False


    def forward(self, x):
        x = self.features(x)
        x = self.output_new(x)
        return x

def train(model, trainloader, device, criterion, optimizer, epochs):
    model.train()
    total_loss = 0.0
    num_samples = 0

    for i, data in enumerate(trainloader):
        start = time.time()


        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        
        loss = criterion(outputs, labels).sum()

        # print(f'output: {outputs}, lables: {labels}')

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        num_samples += labels.size(0)
        end = time.time()

        # print(f'total_loss: {total_loss}, num_samples: {num_samples} time: {end-start}', )
        timecost.append(end-start)
        # print(timecost)
        # i代表什么？
        if (i + 1) % (len(trainloader) // 5) == 0:
            avg_loss = total_loss / num_samples
            los.append(avg_loss)
            print(f'loss: {avg_loss}')

    
# def evaluate_loss(data_iter, net, criterion, devices):
#     l_sum, n = 0.0, 0
#     for features, labels in data_iter:
#         features, labels = features.to(devices), labels.to(devices)
#         outputs = net(features)
#         l = criterion(outputs, labels)
#         l_sum += l.sum()
#         n += labels.numel()
#     return (l_sum / n).to('cpu')


def test(model, testloader, criterion, device):
    model.eval()
    num_samples = 0
    total_loss = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            l = criterion(outputs, labels)
            # print(l)
            l = l.cpu().numpy()
            # print(l)
            total_loss += l.sum()
            num_samples += labels.size(0)
            avg_loss = total_loss / num_samples

    return avg_loss



In [ ]:
device = torch.device('cuda')
model = ResNet34().to(device)
learning_rate = 1e-4
wd = 1e-4
epochs = 20

retrained = True
if retrained == True:
    model.load_state_dict(torch.load('model_resnet34_no_pretrained.ckpt'))

criterion = nn.CrossEntropyLoss(reduction='none')
# optimizer = torch.optim.SGD((param for param in model.parameters()
#                                if param.requires_grad), lr=learning_rate,
#                               momentum=0.9, weight_decay=wd)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for e in range(1, epochs+1):
    print('epoch: {}'.format(e))
    train(model, train_loader, device, criterion, optimizer, epochs)
    accuracy = test(model, train_valid_loader,criterion, device)
    print("test accuracy: {:.2f}".format(accuracy))
    v_los.append(accuracy) 



torch.save(model.state_dict(),'model_resnet34_no_pretrained.ckpt')#保存为model.ckpt

In [12]:
import pandas as pd
df1 = pd.DataFrame(los)
df1.to_csv('loss_resnet34_no_pretrained_6.csv', index=True)
df2 = pd.DataFrame(v_los)
df2.to_csv('valid_los_resnet34_no_pretrained_6.csv', index=True)

In [13]:
# ckpt = torch.load('model_resnet34.ckpt')
# print(ckpt)